In [14]:
import pandas as pd
import numpy as np

# Read and explore data

In [15]:
df = pd.read_csv('data/clickdata.csv', delimiter='\u0001')

In [16]:
df.head()

,epoch_ms,session_id,country_by_ip_address,region_by_ip_address,url_without_parameters,referrer_without_parameters,visitor_recognition_type,useragent,ua_device_class,ua_device_name,...,ua_facebook_device_name,ua_facebook_device_version,ua_facebook_operating_system_name,ua_facebook_operating_system_version,ua_hacker_attackVector,ua_hacker_toolkit,ua_ie_compatibility_version_major,ua_ie_compatibility_name_version_major,ua_carrier,ua_agent_class
0,1520280001034,be73c8d1b836170a21529a1b23140f8e,US,CA,https://www.bol.com/nl/l/nederlandstalige-kuns...,NaN,ANONYMOUS,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,Robot,Google,...,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Robot
1,1520280001590,c24c6637ed7dcbe19ad64056184212a7,US,CA,https://www.bol.com/nl/l/italiaans-natuur-wete...,NaN,ANONYMOUS,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,Robot,Google,...,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Robot
2,1520280002397,ee391655f5680a7bfae0019450aed396,IT,LI,https://www.bol.com/nl/p/nespresso-magimix-ini...,https://www.bol.com/nl/p/nespresso-magimix-ini...,ANONYMOUS,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:47....,Desktop,Linux Desktop,...,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Browser
3,1520280002598,f8c8a696dd37ca88233b2df096afa97f,US,CA,https://www.bol.com/nl/l/nieuwe-engelstalige-o...,NaN,ANONYMOUS,Mozilla/5.0 (compatible; Googlebot/2.1; +http:...,Robot,Google,...,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Robot
4,1520280004428,f8b0c06747b7dd1d53c0932306bd04d6,US,CA,https://www.bol.com/nl/l/nieuwe-actie-avontuur...,NaN,ANONYMOUS,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...,Robot Mobile,Google,...,NaN,NaN,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,Robot Mobile


Each row in 'df' contains a page request in a session.

CSV column definitions:
* epoch_ms: epoch in milliseconds
* session_id: session identifier
* country_by_ip_address: estimated country based on GeoIP lookup
* region_by_ip_address: estimated region based on GeoIP lookup
* url_without_parameters: 
* referrer_without_parameters: 
* visitor_recognition_type: ANONYMOUS, RECOGNIZED (by cookie) or LOGGEDIN
* useragent: the useragent string sent by the webbrowser
* ua_... : we've parsed the useragent field (with yauaa) and expose it as ua_.... see https://github.com/nielsbasjes/yauaa#values-explained
* ua_agent_class: the class determined by yauaa


In [17]:
# clean up different types of missing values
df = df.replace(np.nan, '', regex=True)
df = df.replace('Unknown', '', regex=True)

In [22]:
crawlers = pd.read_csv('labeled_web_crawlers.csv', delimiter='\t')
crawlers

,count,name,type,info
0,18123,Googlebot,NHT-search,NaN
1,1032,AdsBot-Google-Mobile,NHT-other,NaN
2,670,Mediapartners-Google,NHT-other,NaN
3,234,Sogou,NHT-search,NaN
4,208,bingbot,NHT-search,NaN
5,171,Applebot,NHT-other,NaN
6,65,facebookexternalhit,NHT-other,NaN
7,48,WhatsApp,NHT-other,NaN
8,34,UptimeRobot,NHT-other,NaN
9,33,pinterest,NHT-other,NaN


In [62]:
def trafficType(ua): 
    for i, row in crawlers.iterrows():
        if ua.find(row['name']) >= 0:
            return row['type']
    #no crawler? return human
    return 'HT'

In [ ]:
df['traffic_type'] = df['useragent'].map(trafficType)

In [63]:
df['traffic_type']

0        NHT-search
1        NHT-search
2                HT
3        NHT-search
4        NHT-search
5                HT
6                HT
7        NHT-search
8                HT
9                HT
10       NHT-search
11               HT
12       NHT-search
13       NHT-search
14       NHT-search
15       NHT-search
16               HT
17               HT
18       NHT-search
19               HT
20               HT
21        NHT-other
22               HT
23               HT
24       NHT-search
25       NHT-search
26               HT
27               HT
28       NHT-search
29               HT
            ...    
59752            HT
59753            HT
59754    NHT-search
59755            HT
59756            HT
59757            HT
59758    NHT-search
59759    NHT-search
59760            HT
59761            HT
59762            HT
59763            HT
59764            HT
59765    NHT-search
59766    NHT-search
59767    NHT-search
59768            HT
59769            HT
59770            HT
